# Load libs

In [7]:
import os
import requests

import google_auth_oauthlib.flow
import googleapiclient.discovery
import googleapiclient.errors

import pandas as pd
import json
import datetime
import re

pd.set_option('display.max_columns', None)

# Load .env & handle

In [2]:
from dotenv import load_dotenv
load_dotenv()
API_KEY = os.getenv("YOUTUBE_API_KEY")

In [3]:
HANDLE = "@tuki.music_official"

## 1. Lấy thông tin channel + uploads playlist

In [3]:
ch_res = requests.get(
    "https://www.googleapis.com/youtube/v3/channels",
    params={
        "part": "snippet, statistics, contentDetails",
        "forHandle": HANDLE,
        "key": API_KEY
    }
).json()

if not ch_res.get("items"):
    raise SystemExit("Không tìm thấy channel với handle này.")

ch_item = ch_res["items"][0]
channel_id = ch_item["id"]
upload_pl = ch_item["contentDetails"]["relatedPlaylists"]["uploads"]

print("Channel ID:", channel_id)
print("Title:", ch_item["snippet"]["title"])
print("Subs:", ch_item["statistics"].get("subscriberCount"))

Channel ID: UCp74ruKv2sUiI2HqEhH8pRw
Title: tuki.(16)
Subs: 1100000


In [4]:
ch_res

{'kind': 'youtube#channelListResponse',
 'etag': 'gmB4e6iGPQXGYLe40Yrou1HXSa0',
 'pageInfo': {'totalResults': 1, 'resultsPerPage': 5},
 'items': [{'kind': 'youtube#channel',
   'etag': '-Krf-eCpt5_PDvenghECPP5-vDY',
   'id': 'UCp74ruKv2sUiI2HqEhH8pRw',
   'snippet': {'title': 'tuki.(16)',
    'description': '高校二年生 16歳 弾き語り 誰かの心に響きますように。',
    'customUrl': '@tuki.music_official',
    'publishedAt': '2023-07-18T07:32:30.72368Z',
    'thumbnails': {'default': {'url': 'https://yt3.ggpht.com/56pJ8Rhb9L2wXD6sXgkOkFFSp29OfgvVK3GjkpuqSKv0e0bHi5p-p4S6hZbMhyaECRubkdfO0A=s88-c-k-c0x00ffffff-no-rj',
      'width': 88,
      'height': 88},
     'medium': {'url': 'https://yt3.ggpht.com/56pJ8Rhb9L2wXD6sXgkOkFFSp29OfgvVK3GjkpuqSKv0e0bHi5p-p4S6hZbMhyaECRubkdfO0A=s240-c-k-c0x00ffffff-no-rj',
      'width': 240,
      'height': 240},
     'high': {'url': 'https://yt3.ggpht.com/56pJ8Rhb9L2wXD6sXgkOkFFSp29OfgvVK3GjkpuqSKv0e0bHi5p-p4S6hZbMhyaECRubkdfO0A=s800-c-k-c0x00ffffff-no-rj',
      'width': 800,
     

In [8]:
df_channel_dbt = pd.json_normalize(ch_res, sep="__")
df_channel_dbt

,kind,etag,items,pageInfo__totalResults,pageInfo__resultsPerPage
0,youtube#channelListResponse,gmB4e6iGPQXGYLe40Yrou1HXSa0,"[{'kind': 'youtube#channel', 'etag': '-Krf-eCp...",1,5


In [9]:
# Trích xuất dữ liệu
channels = []
for item in ch_res.get("items", []):
    channels.append({
        "channel_id": item["id"],
        "title": item["snippet"]["title"],
        "description": item["snippet"]["description"],
        "customUrl": item["snippet"].get("customUrl"),
        "publishedAt": item["snippet"]["publishedAt"],
        "country": item["snippet"].get("country"),
        "thumbnail_high": item["snippet"]["thumbnails"]["high"]["url"],
        "uploads_playlist": item["contentDetails"]["relatedPlaylists"]["uploads"],
        "subscriberCount": int(item["statistics"].get("subscriberCount", 0)),
        "viewCount": int(item["statistics"].get("viewCount", 0)),
        "videoCount": int(item["statistics"].get("videoCount", 0)),
    })

# Đưa vào DataFrame
df_channels = pd.DataFrame(channels)
df_channels

,channel_id,title,description,customUrl,publishedAt,country,thumbnail_high,uploads_playlist,subscriberCount,viewCount,videoCount
0,UCp74ruKv2sUiI2HqEhH8pRw,tuki.(16),高校二年生 16歳 弾き語り 誰かの心に響きますように。,@tuki.music_official,2023-07-18T07:32:30.72368Z,JP,https://yt3.ggpht.com/56pJ8Rhb9L2wXD6sXgkOkFFS...,UUp74ruKv2sUiI2HqEhH8pRw,1100000,552984374,330
